In [1]:
# imports
import importlib as imp
import geopandas as gpd
import pandas as pd
import functions as f
# debug
imp.reload(f)
import streamlit as st

In [2]:
# check which forest area is along the way
forest_areas_with_bikelanes_wgs84 = gpd.read_parquet('dataset/processed/forest_areas_crossed_by_bikelane_wgs84.parquet')
forest_areas_with_bikelanes_dk = gpd.read_parquet('dataset/processed/forest_areas_crossed_by_bikelane_DK.parquet')
# Step 1: Ensure both GeoDataFrames are in the same CRS

In [3]:
detailed_map = gpd.GeoDataFrame()

### Get a route

In [16]:
import openrouteservice

# Replace 'your_api_key_here' with your actual OpenRouteService API key
client = openrouteservice.Client(key=st.secrets["OPENROUTSERVICE_API"])

# Define your coordinates
coordinates1 = [[11.581726, 55.606109], [11.699829, 55.592143]]  # Sjeland
coordinates4 = [[10.217285, 57.421294], [10.568847, 55.235288]] # super funky long, it will fail (no path)
coordinates5 = [[10.217285, 57.421294], [9.074707, 54.933455]] # this is long as well
coordinates6 = [[10.217285, 57.421294], [10.568847, 55.235288], [9.074707, 54.933455]]
coordinates2 = [[12.473907, 55.717571], [12.561798, 55.695132]] # in Copenhagen
coordinates7 = [[9.582405, 56.394366], [9.497604, 56.356956]]

coordinates = coordinates7

# Make a request to the OpenRouteService API for cycling routes
routes = client.directions(coordinates=coordinates,
                           profile='cycling-regular',
                           format='geojson')

# The response 'routes' will be a GeoJSON object containing the route
gdf_routes_wgs84 = gpd.GeoDataFrame.from_features(routes['features'])
gdf_routes_wgs84.crs = 'EPSG:4326'
gdf_routes_wgs84.explore()

In [ ]:
# add gdf_routes_wgs84 to the detailed map. column color should be "grey"
detailed_map = detailed_map.append(gdf_routes_wgs84, )

### We have to know which areas are checked already

In [49]:
# gdf_denmark = gdf_routes.to_crs(f.DENMARK_CRS)

##### TODO FROM HERE

In [8]:
imp.reload(f)
forest_areas_already_in_the_road_wg84 = f.get_only_areas_which_are_crossed_by_bikelane(forest_areas_with_bikelanes_wgs84, gdf_routes_wgs84)
forest_areas_already_in_the_road_dk = forest_areas_already_in_the_road_wg84.to_crs(f.DENMARK_CRS)

In [9]:
# reproject shortest path

BUFFER = 500
ADD_MORE_AREA_TO_BE_VISITED = 5 # CANT BE MORE THAN 70 - 68 (start and end point)

gdf_routes_dk_crs = gdf_routes_wgs84.to_crs(f.DENMARK_CRS)
# buffer bikelanes
gdf_routes_DK_buffered = f.buffer_gdf(gdf_routes_dk_crs, BUFFER)

print("Number of forest areas already on the path: ", len(forest_areas_already_in_the_road_dk))

# get forest areas along the way
can_be_reached_forest_areas = f.get_only_areas_which_are_crossed_by_bikelane(forest_areas_with_bikelanes_dk, gdf_routes_DK_buffered)
print("Reachabel forest areas: ", len(can_be_reached_forest_areas))
# exttract the already visited forest areas
indices_to_exclude = forest_areas_already_in_the_road_wg84.index
# Drop rows from temp dataframe based on indices_to_exclude
not_visited_forest_areas_dk = can_be_reached_forest_areas.drop(indices_to_exclude)
print("Extra forest areas: ", len(not_visited_forest_areas_dk))
# keep only the X biggest ones
not_visited_forest_areas_dk["area"] = not_visited_forest_areas_dk["geometry"].area
not_visited_forest_areas_dk.sort_values(by="area", ascending=False, inplace=True)
additional_centroids = not_visited_forest_areas_dk.head(ADD_MORE_AREA_TO_BE_VISITED).to_crs(f.CRS).geometry.centroid.to_list()

# add them to the road
# re-calculation

Number of forest areas already on the path:  1
Reachabel forest areas:  6
Extra forest areas:  5


C:\Users\Gergo Gyori\AppData\Local\Temp\ipykernel_17036\4105203072.py:23: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  additional_centroids = not_visited_forest_areas_dk.head(ADD_MORE_AREA_TO_BE_VISITED).to_crs(f.CRS).geometry.centroid.to_list()


In [10]:
imp.reload(f)
additional_centroids_coordinates = f.list_of_points_to_coordinates(additional_centroids)

In [11]:
expanded_coordinates = [coordinates[0]] + additional_centroids_coordinates + [coordinates[1]]

In [12]:
coordinates

[[9.582405, 56.394366], [9.497604, 56.356956]]

In [13]:
print(expanded_coordinates)

[[9.582405, 56.394366], [9.517759708576467, 56.37936401821536], [9.588502627582242, 56.39142943991185], [9.568734776775418, 56.37839853111008], [9.573681083649301, 56.39935213624885], [9.513766152291726, 56.37076457267524], [9.497604, 56.356956]]


In [50]:
print(len(expanded_coordinates))

7


In [17]:

radiuses = [BUFFER] * len(expanded_coordinates)  # 1000 meters for each point

# Make a request to the OpenRouteService API for cycling routes
routes = client.directions(coordinates=expanded_coordinates,
                           profile='cycling-regular',
                           format='geojson',
                           radiuses=radiuses,
                           optimize_waypoints=True,
                           optimized=False)

gdf_routes = gpd.GeoDataFrame.from_features(routes['features'])
gdf_routes.crs = 'EPSG:4326'
gdf_routes.explore()

In [20]:
gdf_routes.geometry.length

C:\Users\Gergo Gyori\AppData\Local\Temp\ipykernel_17036\3986699614.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_routes.geometry.length


0    0.213297
dtype: float64

In [34]:
# check which forest area is along the way
touched_forests.explore()

In [ ]:
# pick the coordinate

In [ ]:
# re-plan